# HW3

Submit via Slack. Due on Tuesday, April 13th, 2020, 6:29pm PST. You may work with one other person.

## TF-IDF

You are an analyst working at McDonalds as a store operations analyst, and charged with identifying areas for improvement for each franchise. Several metropolitan locations have been suffering recently from lower reviews.

Using the **mcdonalds-yelp-negative-reviews.csv** dataset, clean and parse the text reviews. Explain the decisions you make:
- why remove/keep stopwords?
- which stopwords to remove?
- stemming versus lemmatization?
- regex cleaning and substitution?
- adding in custom stopwords?
- what `n` for your `n-grams`?
- which words to collocate together? (optional)

Finally, generate a TF-IDF report that either **visualizes** or explains for a business (non-technical) stakeholder:
* the features your analysis showed that customers cited as reasons for a poor review
* the most common issues identified from your analysis that generated customer dissatisfaction.

Explain to what degree the TF-IDF findings make sense - what are its limitations?

## Product Attribution (Feature Engineering and Regex Practice)

Download the [dataset](https://dso-560-nlp-text-analytics.s3.amazonaws.com/truncated_catalog.csv) from the class S3 bucket (`dso560-nlp-text-analytics`).

In preparation for the group project, our client company has provided a dataset of women's clothing products they are considering cataloging. 

1. Filter for only **women's clothing items**.

2. For each clothing item:

* Identify its **category**:
```
Bottom
One Piece
Shoe
Handbag
Scarf
```
* Identify its **color**:
```
Beige
Black
Blue
Brown
Burgundy
Gold
Gray
Green
Multi 
Navy
Neutral
Orange
Pinks
Purple
Red
Silver
Teal
White
Yellow
```

Your output will be the same dataset, except with **3 additional fields**:
* `is_womens_clothing`
* `product_category`
* `colors`

`colors` should be a list of colors, since it is possible for a piece of clothing to have multiple colors.

# Answers

## TF-IDF

### Load data

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize

df = pd.read_csv('mcdonalds-yelp-negative-reviews.csv', encoding='latin1')
df.head()

### Preliminary regex cleaning

Before any analysis or further processing of data, I performed the following preliminary regex cleaning:

- made all words lowercase
- changed all negative meaning words (doesn't, isn't, hadn't, etc.) to "not" (We may expect a lot of words of this kind since this is a negative review dataset, changing them into "not" preserves their negative meanings while reducing their variations)
- standardized different spellings of McDonalds
- removed punctuations

All of these cleaning were to make it easier later in the analysis to recognize useful words/information without worrying about the variations of words.

In [ ]:
# clean up text
# lowercase everything
df['review'] = df['review'].str.lower()

# change all words such as "doesn't", "hadn't" into "not" to preserve their negative meanings while reducing variations
df['review'] = df['review'].str.replace(r"\b\w+n't\b",'not')

# standardize the spelling of McDonalds
df['review'] = df['review'].str.replace(r"\b(?:mc ?donald(?:s|'s)?|mcds?)\b",'mcdonalds')

# remove punctuations
df['review'] = df['review'].str.replace(r'[^\w\s]', ' ')

df['review']

### Remove stopwords

I removed the stopwords for the following reasons:

1. Our end goal is to identify the most common issues of McDonalds from the negative reviews, but the stopwords are very common words that do not add to our knowledge about McDonald's issues, meaning they will appear as frequently occurring words but with **no valuable information on the meaning of the texts**;

2. By removing stopwords, we can **reduce dimensionality** in our TF-IDF analysis and also reduce computational cost.

What stopwords to remove:

- Other than the common **"english"** stopwords from ```nltk```, I added **"mcdonalds", "absolute", "absolutely", and "really"** as custom stopwords.

- For "mcdonalds", we already know that the dataset is about McDonalds, so making it a stopword will not reduce useful information.

- For words like "absolute", "absolutely", and "really", since this is a negative review dataset, people's reviews may express lots of emotions using these words, so these words only describe the extent or degree of some emotions but do not provide additional information about the actual content, i.e. what is being negatively reviewed.

In [ ]:
# modify stopwords list
stopwords_list = stopwords.words('english')

# knowing this is a review dataset about mcdonalds, we can make "mcdonalds" a stopword for not providing additional info
stopwords_list = stopwords_list + ['mcdonalds']
# additional stopwords: these words only describe the extent/degree of something, do not provide info about the content
stopwords_list = stopwords_list + ['absolute','absolutely','really']

len(stopwords_list)

In [ ]:
# define a function to remove stopwords
def remove_sw(text):
    
    import re
    
    # split sentence into words
    words = word_tokenize(text)
    
    new_words = []
    # remove stopwords
    for w in words:
        if w in stopwords_list:
            continue
        new_words.append(w)
    
    return ' '.join(new_words)

### Further regex cleaning and substitution

After removing stopwords, I performed some more regex cleaning to reduce word variations:

- different spellings/types of "burgers" were changed into "burger"
- words like "aaahhhh" were removed
- words like "noooo" were changed to "no"

In [ ]:
# cleaned review without stopwords
df['cleaned_review'] = df['review'].apply(lambda x: remove_sw(x))

# further clean up text
df['cleaned_review'] = df['cleaned_review'].str.replace(r'\b(?:ham|beef|bacon)? ?burgers?\b','burger')
df['cleaned_review'] = df['cleaned_review'].str.replace(r'\b(?:a+|u+)h+\b','')
df['cleaned_review'] = df['cleaned_review'].str.replace(r'\bn+?o+?\b','no')

df['cleaned_review']

### Lemmatization

I chose to perform lemmatization instead of stemming on the reviews mainly because at the end of the analysis, we want to identify and understand the specific issues of McDonald's, and lemmatization will return the **actual and meaningful words considering the context** rather than the stems only, so it would make it much easier than stemming to understand the tokens/words returned by our final analysis and to summarize McDonald's issues.

In [ ]:
# lemmatization
# reference: https://gist.github.com/gaurav5430/9fce93759eb2f6b1697883c3782f30de#file-nltk-lemmatize-sentences-py
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return ' '.join(lemmatized_sentence)

df['lemma'] = df['cleaned_review'].apply(lambda x: lemmatize_sentence(x))
df['lemma']

### TF-IDF

For the TF-IDF vectorizer, I used **bi- and trigrams (n = 2 and n = 3)** because I believe these two-word and three-word phrases are indicative enough to allow us to understand the meaning of the "local" text without getting too detailed.

I also set the token pattern to only consider words with 3 or more characters and set the maximum occurrences of a token to be 50% of all documents. Too frequently occurring tokens are not unique enough for us to understand the meaning of each document.

In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(2,3),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.5)

X = vectorizer.fit_transform(df['lemma'])
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf

In [ ]:
# add up the tf-idf score of each document for each token and sort in descending order of the token tf-idf scores
tf_idf_terms = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf_terms, columns=['score'])
score.sort_values(by='score', ascending=False, inplace=True)
score

In [ ]:
# visualize the top tokens
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# data
xs = score.head(25).index
ys = score.head(25)['score']

# main plot
fig, ax = plt.subplots(figsize=(17,7))
ax.bar(xs,ys,color='#2d4673',width=0.6)

# x-axis gridline
ax.xaxis.grid(False)

# x,y range
plt.margins(x=0.02)

# x,y ticks
plt.xticks(xs,rotation=45,ha='right',fontsize=17,color='black')
plt.yticks(fontsize=17,color='black')
plt.tick_params(axis="both",which="both",bottom=False,top=False,left=False,right=False)

# x,y labels
plt.ylabel('TF-IDF',fontsize=20,color='black',labelpad=10)
plt.xlabel('Feature terms',fontsize=20,color='black',labelpad=10)

# title
plt.title('Top 25 Feature Terms by TF-IDF Score',fontsize=22)
    
plt.show()

From the visualization above, we can see that customers frequently cited feature terms such as **drive thru, get/take/place order right/wrong, wait minute, and different types of food (ice cream, big mac, chicken nuggets, french fries)** as reasons for giving negative reviews. In the following steps, I would manually look through the documents containing some of these feature terms to further summarize the specific issues.

In [ ]:
df.loc[df['lemma'].str.contains(r'\bdrive thru\b'),'review']

In [ ]:
df.loc[df['lemma'].str.contains(r'\b(?:get|take|place) order (?:right|wrong)?|order (?:right|wrong)\b'),'review']

After manually looking at the documents containing "drive thru" (325 reviews), the common issues I saw include **unclear instructions at the speaker, long waiting time, unfriendly communication (e.g. no smiles), and unprofessional handling of orders (e.g. not wearing gloves and serve with bare hands). "Wrong order"** is another very common issue cited in these reviews, so I also checked the documents containing feature terms related to getting/taking/placing orders right/wrong (241 reviews). Issues related to ordering include **ordering wrong items multiple times, disorganized service, etc.**

For these specific issues, I really believe that McDonald's operations team should **schedule regular employee trainings and evaluations with a system of rewards and penalties.** Training and evaluation areas should include customer communication, order handling, and teamwork/streamline service. Getting the order right should be the most basic requirement for the employees, and it ties into handling orders fast and well. Good communication with customers is also a foundation to take orders correctly or make customers satisfied and happy even when some small mistakes occur occasionally.

### Comments on TF-IDF Analysis



TF-IDF **balances between term frequency and document frequency**, meaning that to have a high TF-IDF score, a token/term needs to appear not in too many documents within the corpus, but when it appears, it appears a lot of times in that document. Given this consideration, I think TF-IDF makes better sense than a pure word-count analysis in this case of analyzing reviews, as it can help to identify the relatively more **important, relevant, and descriptive keywords and issues**.

However, TF-IDF has its own limitations. While it considers both term frequency and document frequency of a token/term, it is still analyzing this single token/term **without capturing its position in text, semantics, and sentiment**. For example, from the TF-IDF analysis above, I knew that "drive thru" is an important and relevant term cited in the reviews, but I still need to manually look through the documents containing "drive thru" to understand the common sentiment and what common issues occur at the drive-thrus.

### Collocations (optional)

I took a look at the top 30 collocating bigrams, and I think that in future analyses, we may combine these words into one token, so when we perform the bi- and trigram TF-IDF analysis again, these tokens would be read together with one or two other words to add to our knowledge about them.

In [ ]:
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures

documents_words = []
for i in range(len(df)):
    documents_words.append(word_tokenize(df.loc[i,'lemma']))
    
len(documents_words)

In [ ]:
collocation_finder = BigramCollocationFinder.from_documents(documents_words)
measures = BigramAssocMeasures()

collocation_finder.nbest(measures.raw_freq, 30)

## Product Attribution (Feature Engineering and Regex Practice)

### Process data

In [ ]:
import pandas as pd

# load data
catalog = pd.read_csv('truncated_catalog.csv')
catalog.head()

In [ ]:
# create a copy of the original dataset
catalog_copy = catalog.copy()

# fill missing values
catalog.fillna('None', inplace=True)

# clean up columns that are helpful for tagging categories and colors
# remove punctuations, \n, etc. to better recognize keywords
for col in ['name', 'description', 'brand_category', 'brand_canonical_url', 'details']:
    catalog[col] = catalog[col].str.replace('\n', ' ')
    catalog[col] = catalog[col].str.replace('\r', ' ')
    catalog[col] = catalog[col].str.replace(r'https:\/\/(?:www\.)?|\.com', '')
    catalog[col] = catalog[col].str.replace(r'[^\w\s]|_', ' ')

# combine useful columns into a single column, so we only need to look for keywords in this single column 
catalog['concat_description'] = catalog['name'] + ' ' + catalog['description'] + ' ' \
                                + catalog['brand_category'] + ' ' + catalog['brand_canonical_url'] + ' ' \
                                + catalog['details']

catalog.head()

### is_womens_clothing

In [ ]:
import re
import numpy as np

# find out whether each item description contains words referring to women's clothing
catalog['is_womens_clothing'] = catalog['concat_description'].str.\
                                contains(
                                r"\b(?:woman|women|girl|female|lady)(?:s|'s)?|ladies|dress(?:es)?|skirts?|purses?\b",
                                flags=re.IGNORECASE)

catalog[['is_womens_clothing']].head()

In [ ]:
# 13172 items are labeled as women's clothing
catalog['is_womens_clothing'].sum()

### product_category

The following method I used to label product categories may seem complicated, but i think it is the method that makes the most sense to me.

- I defined a function that takes into the dataframe, the text column to search through, and the category name (i.e. tag) and would return five boolean columns, one for each category. If a document contains keywords of a category, this category column will have "True" otherwise "False" for this document.

- For all items that contain keywords of exactly one category, I directly labeled them with this category name.

- For items that contain keywords of more than one category, I reran the function using a more specific text column to search for keywords and narrow down their potential categories.

- This process was repeated until all items were assigned with a reasonable category label.

In [ ]:
# define a function to recognize item category
def category_tag(df, col, tag):
    if tag=='Bottom':
        df[tag] = df[col].str.contains(
            r"\b(?:pant|jean|trouser|short|tight)s|(?:skirt|legging|palazzo|skort|wide leg)s?\b",
            flags=re.IGNORECASE)
    elif tag=='One Piece':
        df[tag] = df[col].str.contains(
            r"\b(?:one ?piece|onesie|gown|robe|romper|(?:body|jump|boiler) ?suit|shortall|c?over ?all)s?|dress(?:es)?\b",
            flags=re.IGNORECASE)
    elif tag=='Shoe':
        df[tag] = df[col].str.contains(
        r"\b(?:shoe|sneaker|sandal|heel|boot|slipper|flip ?flop|trainer|platform|oxford|mule|brogue|loafer|moccasin|derby)s?|flats|derbies\b",
        flags=re.IGNORECASE)
    elif tag=='Handbag':
        df[tag] = df[col].str.contains(
            r"\b(?:hand|saddle|bucket|frame) ?bags?|(?:purse|clutch|hobo|tote)(?:es|s)?\b",
            flags=re.IGNORECASE)
    else:
        df[tag] = df[col].str.contains(
            r"\b(?:scar|neckerchie)(?:fs?|ves)|(?:muffler|shawl|pashmina|bandana|stole)s?\b",
            flags=re.IGNORECASE)

In [ ]:
# for each item, find out whether it contains the keywords of each category
for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
    category_tag(catalog, 'concat_description', tag)
    
catalog[['Bottom','One Piece','Shoe','Handbag','Scarf']].head()

In [ ]:
# some items may contain keywords of more than one category, check the sum of the category columns to be sure
catalog['cat_sum'] = catalog[['Bottom','One Piece','Shoe','Handbag','Scarf']].sum(axis=1)
catalog[['cat_sum']].head()

In [ ]:
# first, deal with the items that have keywords in exactly one category or no categories
catalog['product_category'] = np.nan

for i in catalog.loc[catalog['cat_sum']<=1].index:
    for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
        if catalog.loc[i,tag]:
            catalog.loc[i,'product_category'] = tag
            
catalog[['product_category']].head()

In [ ]:
# check the items that contain keywords of more than one category
multi_cat = catalog.loc[catalog['cat_sum']>1,['brand','name','description',
                                              'brand_category','brand_canonical_url',
                                              'details','concat_description']].copy()
multi_cat.shape
# 2289 items have keywords of more than one category, need to treat them specifically

In [ ]:
# "name" is the most accurate column to determine the actual category, rerun tagging using the "name" column
for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
    category_tag(multi_cat, 'name', tag)
multi_cat['cat_sum'] = multi_cat[['Bottom','One Piece','Shoe','Handbag','Scarf']].sum(axis=1)

# recheck items that have keywords of exactly one category
for i in multi_cat.loc[multi_cat['cat_sum']==1].index:
    for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
        if multi_cat.loc[i,tag]:
            multi_cat.loc[i,'product_category'] = tag
            catalog.loc[i,'product_category'] = tag
            
multi_cat[['product_category']].head()

In [ ]:
# 635 more items to tag
multi_cat = multi_cat.loc[multi_cat['product_category'].isnull(),['brand','name','description',
                                                                  'brand_category','brand_canonical_url',
                                                                  'details','concat_description']].copy()
multi_cat.shape

In [ ]:
# use "brand_category" to tag
for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
    category_tag(multi_cat, 'brand_category', tag)
multi_cat['cat_sum'] = multi_cat[['Bottom','One Piece','Shoe','Handbag','Scarf']].sum(axis=1)

for i in multi_cat.loc[multi_cat['cat_sum']==1].index:
    for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
        if multi_cat.loc[i,tag]:
            multi_cat.loc[i,'product_category'] = tag
            catalog.loc[i,'product_category'] = tag
            
multi_cat[['product_category']].head()

In [ ]:
# 352 more items to tag
multi_cat = multi_cat.loc[multi_cat['product_category'].isnull(),['brand','name','description',
                                                                  'brand_category','brand_canonical_url',
                                                                  'details','concat_description']].copy()
multi_cat.shape

In [ ]:
# the last word in the name is usually the actual item
multi_cat['name_last'] = multi_cat['name'].str.split().apply(lambda x: x[-1])

for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
    category_tag(multi_cat, 'name_last', tag)
multi_cat['cat_sum'] = multi_cat[['Bottom','One Piece','Shoe','Handbag','Scarf']].sum(axis=1)

for i in multi_cat.loc[multi_cat['cat_sum']==1].index:
    for tag in ['Bottom','One Piece','Shoe','Handbag','Scarf']:
        if multi_cat.loc[i,tag]:
            multi_cat.loc[i,'product_category'] = tag
            catalog.loc[i,'product_category'] = tag
            
multi_cat[['product_category']].head()

In [ ]:
# 305 more items to tag
multi_cat = multi_cat.loc[multi_cat['product_category'].isnull(),['brand','name','description',
                                                                  'brand_category','brand_canonical_url',
                                                                  'details','concat_description']].copy()
multi_cat.shape

In [ ]:
# remove items in undefined categories - 16 items left
multi_cat['other'] = multi_cat['concat_description'].str.contains(
    r"\b(?:shirt|tee|tshirt|top|sweater|jacket|blouse|coat|sock|polo|belt|blazer|pullover|turtleneck|earring|glove|tank|sweatshirt|necklace|set)s?\b",
    flags=re.IGNORECASE)

multi_cat = multi_cat.loc[multi_cat['other']==False]
multi_cat.shape

In [ ]:
# manually assign categories after manually looking through the descriptions
for i in multi_cat.index:
    
    if multi_cat.loc[i,'brand_category'] == 'dressesandjumpsuits' or \
    'SWIMWEAR' in multi_cat.loc[i,'brand_category'].split() or \
    'FootiesRompers' in multi_cat.loc[i,'brand_category'].split():
        multi_cat.loc[i,'product_category'] = 'One Piece'
        catalog.loc[i,'product_category'] = 'One Piece'
    
    elif 'Pant' in multi_cat.loc[i,'name'].split() or 'pant' in multi_cat.loc[i,'name'].split():
        multi_cat.loc[i,'product_category'] = 'Bottom'
        catalog.loc[i,'product_category'] = 'Bottom'
    
    elif 'Flat' in multi_cat.loc[i,'name'].split():
        multi_cat.loc[i,'product_category'] = 'Shoe'
        catalog.loc[i,'product_category'] = 'Shoe'
    
    elif 'clutch' in multi_cat.loc[i,'description'].split() or \
    'saddlebag' in multi_cat.loc[i,'description'].split() or \
    'purse' in multi_cat.loc[i,'description'].split():
        multi_cat.loc[i,'product_category'] = 'Handbag'
        catalog.loc[i,'product_category'] = 'Handbag'

### colors

In [ ]:
# find all colors in item descriptions
catalog['color_list'] = catalog['concat_description'].str.findall(
    r"\bbeige|light brown|black|blue ?green|blue|brown|umber|burgundy|gold(?:en)?|gray|grey|green|navy|neutral|orange|aurantia|pink|purple|violet|red|scarlet|silver|teal|white|yellow|(?:multi(?:ple)?|several|different|many|more than one) ?colou?rs?\b",
    flags=re.IGNORECASE).apply(lambda x: [w.lower() for w in x]).apply(lambda x: set(x))
# standardize all words to lowercase and make the list a set to avoid repeated colors

# number of unique colors in the set
catalog['n_colors'] = catalog['color_list'].apply(len)
catalog[['color_list','n_colors']].head(10)

In [ ]:
# tag color labels
catalog['colors'] = np.nan

# label items with more than one color as "Multi"
catalog.loc[catalog['n_colors']>1,'colors'] = 'Multi'

# label items with one color
catalog.loc[catalog['n_colors']==1,'colors'] = catalog.loc[catalog['n_colors']==1,'color_list'].\
                                                                        apply(lambda x:list(x)[0])

In [ ]:
# define a function to standardize color labels
def color_tag(text):
    
    color = text
    
    if isinstance(text, str):

        if text == 'beige' or text == 'light brown':
            color = 'Beige'
        elif text == 'blue green' or text == 'bluegreen' or text == 'teal':
            color = 'Teal'
        elif text == 'brown' or text == 'umber':
            color = 'Brown'
        elif text == 'gold' or text == 'golden':
            color = 'Gold'
        elif text == 'gray' or text == 'grey':
            color = 'Gray'
        elif text == 'orange' or text == 'aurantia':
            color = 'Orange'
        elif text == 'purple' or text == 'violet':
            color = 'Purple'
        elif text == 'red' or text == 'scarlet':
            color = 'Red'
        elif re.findall(r"(?:multi(?:ple)?|several|different|many|more than one) ?colou?rs?\b", text) != []:
            color = 'Multi'
        else:
            color = text.capitalize()
    
    return color

catalog['colors'] = catalog['colors'].apply(color_tag)
catalog[['color_list', 'n_colors', 'colors']].head(10)

### Output csv

In [ ]:
# attach the three output columns to the original dataset
for col in ['is_womens_clothing', 'product_category', 'colors']:
    catalog_copy[col] = catalog[col]

catalog_copy.head()

In [ ]:
# export to csv if needed
# catalog_copy.to_csv('catalog_with_category_color_tag.csv', index=False)